# Data Science for Social Justice Workshop: TF-IDF

* * * 

<div class="alert alert-success">  
    
### Learning Objectives 
    
* Understand the TF-IDF algorithm, and how it's used in information retrieval.
* Understand the difference between TF-IDF and simple word counts.
* Use Scikit Learn to extract TF-IDF scores from our text data.
* Use TF-IDF scores to compare posts.
</div>

### Icons Used in This Notebook
🔔 **Question**: A quick question to help you understand what's going on.<br>
🥊 **Challenge**: Interactive excersise. We'll work through these in the workshop!<br>
💡 **Tip**: How to do something a bit more efficiently or effectively.<br>
⚠️ **Warning:** Heads-up about tricky stuff or common mistakes.<br>

### Sections
1. [Term Frequency-Inverse Document Frequency (TF-IDF)](#tfidf)
2. [Reading Data with Pandas](#read)
3. [Dropping Columns and Missing Values](#drop)
4. [Preprocessing Text Data with SpaCy](#clean)
5. [Phrase Modeling with Gensim](#gensim)
6. [Saving Data](#save)

<a id='tfidf'></a>

# Term Frequency-Inverse Document Frequency (TF-IDF)
In the previous notebook, we covered methods to clean and tokenize text into units (like words, bigrams, and trigrams).

For many data science applications, we need a way to convert the content of a text (the list of tokens) into useful numbers that can be used as the features of a model or analysis. One way to do this is to to use a method called Term Frequency-Inverse Document Frequency (TF-IDF). 

This ultimately is a question of **representation**. A text representation is easier for humans to understand, but a numerical representation is easier to perform large scale computational analysis on. The more we change the representation, the more distant our reading of the text becomes. We may gain some benefits from this, but we must also keep in mind what we lose.

## Retrieving the Dataset

We retrieve the data, as we did in the previous notebook:

In [ ]:
import os
import pickle
import pandas as pd

In [ ]:
df = pd.read_csv('../../data/aita_clean.csv')

In [ ]:
with open('../../data/aita_trigrams.pickle', 'rb') as f:
    # Load the object from the file
    aita_trigrams = pickle.load(f)

Note that the documents in our cleaned-up `df` is the same length as the list `aita_trigrams` we created in the previous notebook.

In [ ]:
len(df) == len(aita_trigrams)

## Implementing TF-IDF
TF-IDF, short for **term frequency–inverse document frequency**, is a metric that reflects how important a word is to a **document** in a collection or **corpus**. When talking about text datasets, the dataset is called a corpus, and each datapoint is a document. A document can be a post, a paragraph, a webpage, whatever is considered the individual unit of text for a given datset. A **term** is each unique token in a document (we previously also referred to this as **type**). 

For example in a corpus of sentences, a document might be: `"I went to New York City in New York state."` 

The processed tokens in that document might be: `[went, new_york, city, new_york, state]`.

The document would have four unique terms: `[went, new_york, city, state]`.

The TF-IDF value increases proportionally to the number of times a word appears in the document (the term frequency, or TF), and is offset by the number of documents in the corpus that contain the word (the inverse document frequency, or IDF). This helps to adjust for the fact that some words appear more frequently in general – such as articles and prepositions.

We won't go into much detail about the math behind calculating the TF-IDF (see the D-Lab Text Analysis workshop videos to see more). The key components to remember are:

1. There is one TF-IDF score per unique word and unique document.
2. A high TF-IDF score suggests that word is descriptive of that document.
3. A low TF-IDF score may be because either the word is not frequent in that document, or that it is frequent in many documents in the dataset - either way, it may not be a good descriptor of that document.

The intuition is that if a word occurs many times in one post but rarely in the rest of the corpus, it is probably useful for characterizing that post; conversely, if a word occurs frequently in a post but also occurs frequently in the corpus, it is probably less characteristic of that post.

We will use the [`CountVectorizer()`](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html) from a package called `scikit-learn` (one of the most commonly used machine learning packages in Python) to generate the term-frequency matrix for a given corpus, which is the first part of calculating the tf-idf.

This is a matrix where the rows corresponds to documents columns correspond to terms in the corpus. Each cell gives a count for the frequency of that term in the document (which may be zero). Let's try it on a toy dataset. Conveniently, there is a built-in tokenizer in `CountVectorizer()` that we will use for now to preprocess the text.

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

corpus = [
  'My cat has paws.',
  'Can we let the dog out?',
  'Our dog really likes the cat but the cat does not agree.']
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(corpus)
pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names_out())
# Use this if your scikit-learn is older
# pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names())

Each column in the matrix represents a unique word in the vocabulary, while each row represents the document in our dataset. In this case, we have three sentences (i.e. the document), and therefore we three rows. The values in each cell are the word counts. Note that with this representation, counts of some words could be 0 if the word did not appear in the corresponding document. 


Lets look at the shape of the matrix. How many unique words are there in the matrix?'

In [ ]:
X.shape

Now, we have numbers representing the contents of the documents! Matrices like this are the simplest way to represent texts. However, it biases most frequent words and ends up ignoring rare words which could have helped is in processing our data more efficiently.

Often, we not only want to focus on the frequency of words present in the corpus but also want to know the importance of the words. This is where TF-IDF (term frequency-inverse document frequency) comes in. TF-IDF is a statistical measure that evaluates how relevant a word is to a document in a collection of documents.

This is done by combining the **term frequency** calculated above and the **inverse document frequency** of the word across a set of documents. For the latter, we divide the total number of documents by the number of documents containing the term, and then take the logarithm of that quotient. (This is calculated for us by the computer). This tells us if a word is common or rare across all documents.

The combination of these two metrics into the TF-IDF will identify how unique a term is to that document.

**Note**: Word order is not retained in this type of featurization, since all that is counted is overall frequency of a word in a document. This is called a **bag-of-words** approach, and significantly simplifies the representation problem, but has been found to be effective in capturing key features of documents.

### Testing TF-IDF with a toy dataset

Let's try TF-IDF out with a toy dataset. Here we have three documents about Python, but with different meanings. If we are trying to distinguish between these documents, the word "Python" would not be very useful, since it occurs in all of the documents, but other terms might, like "Monty", "snake", etc.

In [ ]:
document1 = """Python is a 2000 made-for-TV horror movie directed by Richard
Clabaugh. The film features several cult favorite actors, including William
Zabka of The Karate Kid fame, Wil Wheaton, Casper Van Dien, Jenny McCarthy,
Keith Coogan, Robert Englund (best known for his role as Freddy Krueger in the
A Nightmare on Elm Street series of films), Dana Barron, David Bowe, and Sean
Whalen."""

document2 = """Python, from the Greek word (πύθων/πύθωνας), is a genus of
nonvenomous pythons[2] found in Africa and Asia. Currently, 7 species are
recognised.[2] A member of this genus, P. reticulatus, is among the longest
snakes known."""

document3 = """Monty Python (also collectively known as the Pythons) are a British 
surreal comedy group who created the sketch comedy television show Monty Python's 
Flying Circus, which first aired on the BBC in 1969. Forty-five episodes were made 
over four series."""

document4 = """Python is an interpreted, high-level, general-purpose programming language. 
Created by Guido van Rossum and first released in 1991, Python's design philosophy emphasizes 
code readability with its notable use of significant whitespace. Its language constructs and 
object-oriented approach aim to help programmers write clear, logical code for small and 
large-scale projects."""

document5 = """The Colt Python is a .357 Magnum caliber revolver formerly
manufactured by Colt's Manufacturing Company of Hartford, Connecticut.
It is sometimes referred to as a "Combat Magnum". It was first introduced
in 1955, the same year as Smith &amp; Wesson's M29 .44 Magnum. The now discontinued
Colt Python targeted the premium revolver market segment."""

document6 = """The Pythonidae, commonly known simply as pythons, from the Greek word python 
(πυθων), are a family of nonvenomous snakes found in Africa, Asia, and Australia. 
Among its members are some of the largest snakes in the world. Eight genera and 31
species are currently recognized."""

test_list = [document1, document2, document3, document4, document5, document6]

Let's `CountVectorizer()` again, and customize some parameters. Using `max_df` (max document frequency) we can get rid of words that appear in more than 85% of the corpus, and using `stop_words` we can insert an English stopword list to leave out of the calculations as well.

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(max_df=0.85, stop_words='english')
word_count_vector = cv.fit_transform(test_list)
pd.DataFrame(word_count_vector.toarray(), columns=cv.get_feature_names_out())
# Use this if your scikit-learn is older
# pd.DataFrame(word_count_vector.toarray(), columns=cv.get_feature_names())

How many documents and unique words are in this dataset?

## Using `TfidfTransformer`

Next, we need to compute the inverse document frequency values. We'll call [`tfidf_transformer.fit()`](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfTransformer.html) on the word counts we computed earlier.


In [ ]:
from sklearn.feature_extraction.text import TfidfTransformer

tfidf_transformer = TfidfTransformer() 
tfidf_transformer.fit(word_count_vector)

To get a glimpse of how the IDF values look, let's put these into a DataFrame and sort by weights. Remember, a low IDF indicates something that is less unique.

In [ ]:
# Print IDF values 
df_idf = pd.DataFrame(tfidf_transformer.idf_, index=cv.get_feature_names_out(), columns=["idf_weights"]) 
# Sort ascending 
df_idf.sort_values(by=['idf_weights'])

Notice that the words "python" and "in" have the lowest IDF values. This is expected: these words appear in each and every document in our collection. The lower the IDF value of a word, the less unique it is to any particular document.

Now that we have the idf values, we can compute the TF-IDF scores for our set of documents using `.transform()`

In [ ]:
tf_idf_vector = tfidf_transformer.transform(word_count_vector)

By invoking `tfidf_transformer.transform()` we are computing the TF-IDF scores for our docs. Internally, this is weighting TF scores by their IDF scores, so that the more unique a word, the more its frequency counts in a given document.

Let’s print the TF-IDF values of the first document to see if it makes sense. We place the TF-IDF scores from the third document into a `pandas` data frame and sort it in descending order of scores. We can replace the index in `tf_idf_vector[]` to select a different document to check.

In [ ]:
feature_names = cv.get_feature_names_out() 
  
# Print the scores 
df = pd.DataFrame(tf_idf_vector[2].T.todense(), index=feature_names, columns=["tfidf"]) 
df.sort_values(by=["tfidf"], ascending=False)

What are the most distinctive words for document 3 (Highest TF-IDF scores?) Does it made sense given the document and corpus?

## Using TF-IDF on Reddit datasets

Now that we have a good grasp of how TF-IDF is calculated, let's perform this method on our Reddit data. This time, to save time, we will be using `scikit-learn`'s [`TfidfVectorizer`](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html?highlight=tfidfvectorizer#sklearn.feature_extraction.text.TfidfVectorizer). It is a class that basically allows us to create a matrix of word counts (what we just did with `CountVectorizer`), and immediately transform them into TF-IDF values.

We simply instantiate an object of the `TfidfVectorizer`. Then, we run it by applying the `fit_transform()` method to our two-document corpus `aita_tokens`.

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Settings that you use for count vectorizer will go here
tfidf_vectorizer = TfidfVectorizer(max_df=0.85,
                                   decode_error='ignore',
                                   stop_words='english',
                                   smooth_idf=True,
                                   use_idf=True)

# Fit and transform the texts
tfidf = tfidf_vectorizer.fit_transform(aita_trigrams)

Let's have a look at some of the TF-IDF values:

In [ ]:
# Place TF-IDF values in a DataFrame
tfidf_df = pd.DataFrame(tfidf.todense(), columns=tfidf_vectorizer.get_feature_names_out().ravel())

In [ ]:
tfidf_df.head()

In [ ]:
# Highest TF-IDF values across documents
tfidf_df.sum().sort_values(ascending=False)

## Using TF-IDF to find Similar Posts

We can also use TF-IDF to work out the similarity between any pair of documents. So given one post or comment, we could see which posts or comments are most similar. This can be useful if you're trying to find other examples of a pattern you have found and want to explore further.

This time, our "documents" will not be entire subreddits, but posts/submissions within one subreddit. Let's import the submissions and run the vectorizer without the preprocessing and lemmatizing. Tf-idf will still work this way, and this way, we will be able to read our posts.

We'll use the TF-IDF vectorized output from the previous section. Let's choose a particular document, and try and find similar documents.

In [ ]:
doc_idx = 25

In [ ]:
df['selftext'].iloc[doc_idx]

Let's have a quick look at the TF-IDF scores for the words in this submission to see if these words are indeed typical for this particular submission. Do the distinctive words have to do with the topic of the post?

In [ ]:
tfidf_df.loc[doc_idx].sort_values(ascending=False)

Now let's find the closest posts to this one. The fact that our documents are now in a vector space allows us to make use of mathematical similarity metrics.

**Cosine similarity** is one metric used to measure how similar the documents are irrespective of their size. Mathematically, it measures the cosine of the angle between two vectors projected in a multi-dimensional space. It is equal to 1 if the documents are the same, and decreases to 0 the more dissimilar they are.

We can use a cosine similarity function from `sklearn` to calculate the cosine similarity between each pair of documents:

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
similarities = cosine_similarity(tfidf)
similarities.shape

We can put the text and scores in a dataframe, and sort by the score:

In [ ]:
similar_df = pd.DataFrame({
    'text': df['selftext'].values,
    'score': similarities[doc_idx]}).sort_values('score', ascending=False)

The top document will be the document itself (it's going to have a similarity of 1 with itself). So we look at the next document - does it seem similar?

In [ ]:
similar_df['text'].iloc[0]

In [ ]:
similar_df['text'].iloc[1]

<div class="alert alert-success">

## ❗ Key Points

* Term Frequency, or TF, reflects how often a unique token appears in a corpus.
* Inverse Document Frequency (IDF) reflects the number of documents in the corpus that contain a term. 
* The TF-IDF score of a word reflects how important that word is to a document in a collection or corpus.
* Methods from `scikit-learn` can be used to generate TF and TF-IDF scores for a given corpus.
* TF-IDF scores can be used to calculate how similar documents are. We can do this with mathematical similarity metrics, such as cosine similarity.
    
</div>